- doc2vec.csv

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/학교/Dissertation/Features/text.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df

,title,page_id,text,text_tags,quality
0,Mayan languages,182013,The Mayan languages form a language family spo...,{{short description|Language family spoken in ...,FA
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",{{Short description|Founder of the Umayyad Cal...,FA
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,{{short description|1943 novel by Ayn Rand}}\n...,FA
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,{{Short description|Migratory duck that breeds...,FA
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,{{Short description|World War II Allied nuclea...,FA
...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,{{short description|Former political party in ...,Stub
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",{{Short description|Figures in Australian Abor...,Stub
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,{{Short description|Drug used for treatment of...,Stub
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,{{Use mdy dates|date=July 2023}}\n{{Infobox se...,Stub


In [ ]:
df = df[['title', 'page_id', 'text', 'quality']]
df

,title,page_id,text,quality
0,Mayan languages,182013,The Mayan languages form a language family spo...,FA
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",FA
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,FA
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,FA
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,FA
...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,Stub
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",Stub
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,Stub
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,Stub


In [ ]:
df['quality2'] = np.where((df['quality']=='FA')|(df['quality']=='GA'), 1, 0)
df

,title,page_id,text,quality,quality2
0,Mayan languages,182013,The Mayan languages form a language family spo...,FA,1
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",FA,1
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,FA,1
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,FA,1
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,FA,1
...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,Stub,0
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",Stub,0
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,Stub,0
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,Stub,0


In [ ]:
df.drop(['quality'], axis=1, inplace=True)

In [ ]:
df.rename(columns={'quality2': 'quality'}, inplace=True)

In [ ]:
df

,title,page_id,text,quality
0,Mayan languages,182013,The Mayan languages form a language family spo...,1
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",1
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,1
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,1
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,1
...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,0
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",0
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,0
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,0


In [ ]:
df['quality'].value_counts()

quality
1    2600
0    2600
Name: count, dtype: int64

### Preprocessing

In [ ]:
def doc2vec_preprocessing(text):
    # Split the text by '== References ==' and similar titles, then take the first part
    text_split = re.split(r'==\s*(?:See also|References|Notes|Footnotes|Sources|External links|Works cited|Further reading)\s*==', text, maxsplit=1)[0]

    # Remove '\n' and '\t'
    text_cleaned = re.sub(r'\n+', ' ', text_split)
    text_cleaned = re.sub(r'\t+', ' ', text_cleaned)

    # Convert to lowercase
    text_cleaned = text_cleaned.lower()

    # Remove punctuation
    text_cleaned = text_cleaned.translate(str.maketrans('', '', string.punctuation))

    # Remove numbers
    text_cleaned = re.sub(r'\d+', '', text_cleaned)

    # Tokenize text
    words = word_tokenize(text_cleaned)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Lemmatize words
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]

    # Join words back into a single string
    processed_text = ' '.join(words)

    return processed_text

In [ ]:
df['text_pre'] = df['text'].apply(doc2vec_preprocessing)
df

,title,page_id,text,quality,text_pre
0,Mayan languages,182013,The Mayan languages form a language family spo...,1,mayan language form language family spoken mes...
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",1,muawiya arabic معاوية بن أبي سفيان romanized m...
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,1,fountainhead novel russianamerican author ayn ...
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,1,pintail northern pintail ana acuta duck specie...
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,1,manhattan project research development program...
...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,0,party democratic kampuchea political party cam...
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",0,aboriginal mythology minawara multultu legenda...
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,0,theophyllineephedrahydroxyzine trade name mara...
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,0,channel lake unincorporated community censusde...


### doc2vec

In [ ]:
# Preprocess the text (tokenization)
df['doc2vec_20'] = df['text_pre'].apply(lambda x: word_tokenize(x))

# Create TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['doc2vec_20'])]

# Train a Doc2Vec model
model = Doc2Vec(documents, vector_size=20, min_count=1, workers=4)

# Infer vectors for each document in the DataFrame
df['doc2vec_20'] = df['doc2vec_20'].apply(lambda x: model.infer_vector(x))
df

,title,page_id,text,quality,text_pre,doc2vec_20
0,Mayan languages,182013,The Mayan languages form a language family spo...,1,mayan language form language family spoken mes...,"[0.052285753, -0.7528272, -3.2930627, 7.814262..."
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",1,muawiya arabic معاوية بن أبي سفيان romanized m...,"[1.880606, -4.431403, -2.7955983, -1.8294932, ..."
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,1,fountainhead novel russianamerican author ayn ...,"[-1.1044517, -2.747504, -3.3631063, -5.7099323..."
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,1,pintail northern pintail ana acuta duck specie...,"[-1.9572401, 1.4564344, -2.165325, 6.393158, 1..."
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,1,manhattan project research development program...,"[5.3973994, -3.428697, -5.9390984, 0.7157332, ..."
...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,0,party democratic kampuchea political party cam...,"[0.2985203, 0.7032455, -0.037076205, 0.1380686..."
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",0,aboriginal mythology minawara multultu legenda...,"[-0.050115485, 0.33314192, -0.47374865, 0.3750..."
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,0,theophyllineephedrahydroxyzine trade name mara...,"[-0.0061284443, 0.23356082, -0.22676685, 0.370..."
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,0,channel lake unincorporated community censusde...,"[-0.026024597, -0.11023841, -0.6295018, 0.4262..."


In [ ]:
# Preprocess the text (tokenization)
df['doc2vec_50'] = df['text_pre'].apply(lambda x: word_tokenize(x))

# Create TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['doc2vec_50'])]

# Train a Doc2Vec model
model = Doc2Vec(documents, vector_size=50, min_count=1, workers=4)

# Infer vectors for each document in the DataFrame
df['doc2vec_50'] = df['doc2vec_50'].apply(lambda x: model.infer_vector(x))
df

,title,page_id,text,quality,text_pre,doc2vec_20,doc2vec_50
0,Mayan languages,182013,The Mayan languages form a language family spo...,1,mayan language form language family spoken mes...,"[0.052285753, -0.7528272, -3.2930627, 7.814262...","[-3.1167188, -0.51315886, -4.0492363, 3.337615..."
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",1,muawiya arabic معاوية بن أبي سفيان romanized m...,"[1.880606, -4.431403, -2.7955983, -1.8294932, ...","[2.180839, -2.1110299, -6.7079535, 5.2487364, ..."
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,1,fountainhead novel russianamerican author ayn ...,"[-1.1044517, -2.747504, -3.3631063, -5.7099323...","[-4.2395306, -6.4100766, 1.9549298, 2.377344, ..."
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,1,pintail northern pintail ana acuta duck specie...,"[-1.9572401, 1.4564344, -2.165325, 6.393158, 1...","[-0.9041298, -0.6752747, -3.3694968, -0.584405..."
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,1,manhattan project research development program...,"[5.3973994, -3.428697, -5.9390984, 0.7157332, ...","[-6.8237815, -3.9645262, 3.3373182, 1.5050268,..."
...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,0,party democratic kampuchea political party cam...,"[0.2985203, 0.7032455, -0.037076205, 0.1380686...","[-1.7322543, 0.20272018, -0.98451257, 0.647002..."
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",0,aboriginal mythology minawara multultu legenda...,"[-0.050115485, 0.33314192, -0.47374865, 0.3750...","[-0.7812953, -0.4269692, -0.72439814, -0.08999..."
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,0,theophyllineephedrahydroxyzine trade name mara...,"[-0.0061284443, 0.23356082, -0.22676685, 0.370...","[-0.35274357, -0.093490124, -0.23601085, 0.043..."
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,0,channel lake unincorporated community censusde...,"[-0.026024597, -0.11023841, -0.6295018, 0.4262...","[-0.90208524, -0.48854405, -0.86469644, -0.067..."


In [ ]:
# Preprocess the text (tokenization)
df['doc2vec_100'] = df['text_pre'].apply(lambda x: word_tokenize(x))

# Create TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['doc2vec_100'])]

# Train a Doc2Vec model
model = Doc2Vec(documents, vector_size=50, min_count=1, workers=4)

# Infer vectors for each document in the DataFrame
df['doc2vec_100'] = df['doc2vec_100'].apply(lambda x: model.infer_vector(x))
df

,title,page_id,text,quality,text_pre,doc2vec_20,doc2vec_50,doc2vec_100
0,Mayan languages,182013,The Mayan languages form a language family spo...,1,mayan language form language family spoken mes...,"[0.052285753, -0.7528272, -3.2930627, 7.814262...","[-3.1167188, -0.51315886, -4.0492363, 3.337615...","[-2.94301, -0.8504152, -2.0068648, 3.4947474, ..."
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",1,muawiya arabic معاوية بن أبي سفيان romanized m...,"[1.880606, -4.431403, -2.7955983, -1.8294932, ...","[2.180839, -2.1110299, -6.7079535, 5.2487364, ...","[1.7407136, -0.80208826, -6.6003428, 5.1009145..."
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,1,fountainhead novel russianamerican author ayn ...,"[-1.1044517, -2.747504, -3.3631063, -5.7099323...","[-4.2395306, -6.4100766, 1.9549298, 2.377344, ...","[-4.329825, -7.180442, 1.6074463, 2.195198, -3..."
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,1,pintail northern pintail ana acuta duck specie...,"[-1.9572401, 1.4564344, -2.165325, 6.393158, 1...","[-0.9041298, -0.6752747, -3.3694968, -0.584405...","[-2.6140962, 0.08929471, -2.456376, 0.8568511,..."
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,1,manhattan project research development program...,"[5.3973994, -3.428697, -5.9390984, 0.7157332, ...","[-6.8237815, -3.9645262, 3.3373182, 1.5050268,...","[-6.186118, -2.2152514, 1.7148697, 0.09913541,..."
...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,0,party democratic kampuchea political party cam...,"[0.2985203, 0.7032455, -0.037076205, 0.1380686...","[-1.7322543, 0.20272018, -0.98451257, 0.647002...","[-1.2276016, -0.07646108, -0.23301132, 0.91428..."
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",0,aboriginal mythology minawara multultu legenda...,"[-0.050115485, 0.33314192, -0.47374865, 0.3750...","[-0.7812953, -0.4269692, -0.72439814, -0.08999...","[-0.69870734, -0.33438665, -0.58722293, -0.060..."
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,0,theophyllineephedrahydroxyzine trade name mara...,"[-0.0061284443, 0.23356082, -0.22676685, 0.370...","[-0.35274357, -0.093490124, -0.23601085, 0.043...","[-0.4175495, -0.09883414, -0.20222571, 0.06971..."
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,0,channel lake unincorporated community censusde...,"[-0.026024597, -0.11023841, -0.6295018, 0.4262...","[-0.90208524, -0.48854405, -0.86469644, -0.067...","[-0.8460111, -0.308263, -0.4235552, -0.0420611..."


In [ ]:
# Preprocess the text (tokenization)
df['doc2vec_150'] = df['text_pre'].apply(lambda x: word_tokenize(x))

# Create TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['doc2vec_150'])]

# Train a Doc2Vec model
model = Doc2Vec(documents, vector_size=50, min_count=1, workers=4)

# Infer vectors for each document in the DataFrame
df['doc2vec_150'] = df['doc2vec_150'].apply(lambda x: model.infer_vector(x))
df

,title,page_id,text,quality,text_pre,doc2vec_20,doc2vec_50,doc2vec_100,doc2vec_150
0,Mayan languages,182013,The Mayan languages form a language family spo...,1,mayan language form language family spoken mes...,"[0.052285753, -0.7528272, -3.2930627, 7.814262...","[-3.1167188, -0.51315886, -4.0492363, 3.337615...","[-2.94301, -0.8504152, -2.0068648, 3.4947474, ...","[-1.2495561, -2.6074727, -3.1147242, 6.7186394..."
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",1,muawiya arabic معاوية بن أبي سفيان romanized m...,"[1.880606, -4.431403, -2.7955983, -1.8294932, ...","[2.180839, -2.1110299, -6.7079535, 5.2487364, ...","[1.7407136, -0.80208826, -6.6003428, 5.1009145...","[2.150459, -1.2366129, -6.3890724, 6.032428, -..."
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,1,fountainhead novel russianamerican author ayn ...,"[-1.1044517, -2.747504, -3.3631063, -5.7099323...","[-4.2395306, -6.4100766, 1.9549298, 2.377344, ...","[-4.329825, -7.180442, 1.6074463, 2.195198, -3...","[-4.9335074, -7.4910164, 0.34299594, 2.2856274..."
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,1,pintail northern pintail ana acuta duck specie...,"[-1.9572401, 1.4564344, -2.165325, 6.393158, 1...","[-0.9041298, -0.6752747, -3.3694968, -0.584405...","[-2.6140962, 0.08929471, -2.456376, 0.8568511,...","[-1.4054794, 0.1011202, -3.005766, 0.24375506,..."
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,1,manhattan project research development program...,"[5.3973994, -3.428697, -5.9390984, 0.7157332, ...","[-6.8237815, -3.9645262, 3.3373182, 1.5050268,...","[-6.186118, -2.2152514, 1.7148697, 0.09913541,...","[-5.779205, -3.8620083, 3.9781356, 0.46458927,..."
...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,0,party democratic kampuchea political party cam...,"[0.2985203, 0.7032455, -0.037076205, 0.1380686...","[-1.7322543, 0.20272018, -0.98451257, 0.647002...","[-1.2276016, -0.07646108, -0.23301132, 0.91428...","[-1.2699313, 0.07084496, -0.6724753, 0.9832287..."
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",0,aboriginal mythology minawara multultu legenda...,"[-0.050115485, 0.33314192, -0.47374865, 0.3750...","[-0.7812953, -0.4269692, -0.72439814, -0.08999...","[-0.69870734, -0.33438665, -0.58722293, -0.060...","[-0.6758598, -0.40375656, -0.60720843, -0.1228..."
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,0,theophyllineephedrahydroxyzine trade name mara...,"[-0.0061284443, 0.23356082, -0.22676685, 0.370...","[-0.35274357, -0.093490124, -0.23601085, 0.043...","[-0.4175495, -0.09883414, -0.20222571, 0.06971...","[-0.31140506, -0.14326145, -0.28372833, 0.0403..."
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,0,channel lake unincorporated community censusde...,"[-0.026024597, -0.11023841, -0.6295018, 0.4262...","[-0.90208524, -0.48854405, -0.86469644, -0.067...","[-0.8460111, -0.308263, -0.4235552, -0.0420611...","[-0.8384187, -0.43765557, -0.6848947, -0.02314..."


In [ ]:
# Preprocess the text (tokenization)
df['doc2vec_200'] = df['text_pre'].apply(lambda x: word_tokenize(x))

# Create TaggedDocument
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['doc2vec_200'])]

# Train a Doc2Vec model
model = Doc2Vec(documents, vector_size=50, min_count=1, workers=4)

# Infer vectors for each document in the DataFrame
df['doc2vec_200'] = df['doc2vec_200'].apply(lambda x: model.infer_vector(x))
df

,title,page_id,text,quality,text_pre,doc2vec_20,doc2vec_50,doc2vec_100,doc2vec_150,doc2vec_200
0,Mayan languages,182013,The Mayan languages form a language family spo...,1,mayan language form language family spoken mes...,"[0.052285753, -0.7528272, -3.2930627, 7.814262...","[-3.1167188, -0.51315886, -4.0492363, 3.337615...","[-2.94301, -0.8504152, -2.0068648, 3.4947474, ...","[-1.2495561, -2.6074727, -3.1147242, 6.7186394...","[-1.0646585, -1.3346218, -3.5811057, 6.087087,..."
1,Mu'awiya I,207068,"Mu'awiya I (Arabic: معاوية بن أبي سفيان, roman...",1,muawiya arabic معاوية بن أبي سفيان romanized m...,"[1.880606, -4.431403, -2.7955983, -1.8294932, ...","[2.180839, -2.1110299, -6.7079535, 5.2487364, ...","[1.7407136, -0.80208826, -6.6003428, 5.1009145...","[2.150459, -1.2366129, -6.3890724, 6.032428, -...","[2.227598, -1.8654984, -6.8027906, 5.964404, -..."
2,The Fountainhead,180464,The Fountainhead is a 1943 novel by Russian-Am...,1,fountainhead novel russianamerican author ayn ...,"[-1.1044517, -2.747504, -3.3631063, -5.7099323...","[-4.2395306, -6.4100766, 1.9549298, 2.377344, ...","[-4.329825, -7.180442, 1.6074463, 2.195198, -3...","[-4.9335074, -7.4910164, 0.34299594, 2.2856274...","[-5.734447, -8.045718, 0.11687499, 2.7109966, ..."
3,Northern pintail,218361,The pintail or northern pintail (Anas acuta) i...,1,pintail northern pintail ana acuta duck specie...,"[-1.9572401, 1.4564344, -2.165325, 6.393158, 1...","[-0.9041298, -0.6752747, -3.3694968, -0.584405...","[-2.6140962, 0.08929471, -2.456376, 0.8568511,...","[-1.4054794, 0.1011202, -3.005766, 0.24375506,...","[-1.713172, 0.1169865, -4.2138796, 0.089964174..."
4,Manhattan Project,19603,The Manhattan Project was a research and devel...,1,manhattan project research development program...,"[5.3973994, -3.428697, -5.9390984, 0.7157332, ...","[-6.8237815, -3.9645262, 3.3373182, 1.5050268,...","[-6.186118, -2.2152514, 1.7148697, 0.09913541,...","[-5.779205, -3.8620083, 3.9781356, 0.46458927,...","[-7.4099584, -3.9145057, 1.6065525, 0.99902433..."
...,...,...,...,...,...,...,...,...,...,...
5195,Party of Democratic Kampuchea,265468,The Party of Democratic Kampuchea was a politi...,0,party democratic kampuchea political party cam...,"[0.2985203, 0.7032455, -0.037076205, 0.1380686...","[-1.7322543, 0.20272018, -0.98451257, 0.647002...","[-1.2276016, -0.07646108, -0.23301132, 0.91428...","[-1.2699313, 0.07084496, -0.6724753, 0.9832287...","[-1.7050859, 0.38429388, -0.6107001, 0.8276098..."
5196,Minawara and Multultu,95240,"In Aboriginal mythology, Minawara and Multultu...",0,aboriginal mythology minawara multultu legenda...,"[-0.050115485, 0.33314192, -0.47374865, 0.3750...","[-0.7812953, -0.4269692, -0.72439814, -0.08999...","[-0.69870734, -0.33438665, -0.58722293, -0.060...","[-0.6758598, -0.40375656, -0.60720843, -0.1228...","[-0.8250788, -0.45138186, -0.5446994, -0.07432..."
5197,Theophylline/ephedra/hydroxyzine,262652,Theophylline/ephedra/hydroxyzine (trade name M...,0,theophyllineephedrahydroxyzine trade name mara...,"[-0.0061284443, 0.23356082, -0.22676685, 0.370...","[-0.35274357, -0.093490124, -0.23601085, 0.043...","[-0.4175495, -0.09883414, -0.20222571, 0.06971...","[-0.31140506, -0.14326145, -0.28372833, 0.0403...","[-0.35810485, -0.14381687, -0.19762194, 0.0832..."
5198,"Channel Lake, Illinois",111450,Channel Lake is an unincorporated community an...,0,channel lake unincorporated community censusde...,"[-0.026024597, -0.11023841, -0.6295018, 0.4262...","[-0.90208524, -0.48854405, -0.86469644, -0.067...","[-0.8460111, -0.308263, -0.4235552, -0.0420611...","[-0.8384187, -0.43765557, -0.6848947, -0.02314...","[-1.0711137, -0.5365713, -0.7069544, -0.135931..."


In [ ]:
df.to_csv('/content/drive/MyDrive/학교/Dissertation/Data Preprocessing/doc2vec.csv')

### Modelling

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from xgboost import XGBClassifier

In [ ]:
def train_and_evaluate_models(X, y, cv=5, test_size=0.2):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, stratify=y)

    # Initialize the models
    models = {
        'Logistic Regression': LogisticRegression(),
        'SVM': SVC(),
        'Random Forest': RandomForestClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'XGBoost': XGBClassifier()
    }

    # Initialize a list to store the results
    results = []

    # Define custom scoring
    scoring = {
        'accuracy': make_scorer(accuracy_score),
        'precision': make_scorer(precision_score, average='weighted'),
        'recall': make_scorer(recall_score, average='weighted'),
        'f1': make_scorer(f1_score, average='weighted')
    }

    # StratifiedKFold for cross-validation
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    # Loop through the models
    for model_name, model in models.items():
        # Perform cross-validation on the training set
        cv_results = {metric: cross_val_score(model, X_train, y_train, cv=skf, scoring=scorer).mean() for metric, scorer in scoring.items()}

        # Train the model on the full training set
        model.fit(X_train, y_train)

        # Predict the test set
        y_pred = model.predict(X_test)

        # Calculate the metrics on the test set
        test_accuracy = accuracy_score(y_test, y_pred)
        test_precision = precision_score(y_test, y_pred, average='weighted')
        test_recall = recall_score(y_test, y_pred, average='weighted')
        test_f1 = f1_score(y_test, y_pred, average='weighted')

        # Append the results to the list
        results.append({
            'Model': model_name,
            'CV Accuracy': cv_results['accuracy'],
            'CV Precision': cv_results['precision'],
            'CV Recall': cv_results['recall'],
            'CV F1 Score': cv_results['f1'],
            'Test Accuracy': test_accuracy,
            'Test Precision': test_precision,
            'Test Recall': test_recall,
            'Test F1 Score': test_f1
        })

    # Create a dataframe from the results
    results_df = pd.DataFrame(results)

    return results_df

In [ ]:
X = list(df['doc2vec_20'])
y = df['quality']

In [ ]:
doc2vec_20 = train_and_evaluate_models(X, y)
doc2vec_20

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.792067,0.792661,0.792067,0.791959,0.796154,0.796171,0.796154,0.796151
1,SVM,0.837500,0.839914,0.837500,0.837211,0.834615,0.836813,0.834615,0.834345
2,Random Forest,0.826923,0.827450,0.826683,0.823100,0.832692,0.838140,0.832692,0.832016
3,AdaBoost,0.786058,0.786630,0.786058,0.785955,0.791346,0.791356,0.791346,0.791344
4,XGBoost,0.820673,0.822274,0.820673,0.820453,0.836538,0.839193,0.836538,0.836218


In [ ]:
X = list(df['doc2vec_50'])
y = df['quality']

In [ ]:
doc2vec_50 = train_and_evaluate_models(X, y)
doc2vec_50

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.807452,0.807952,0.807452,0.807372,0.825962,0.825992,0.825962,0.825958
1,SVM,0.839423,0.841499,0.839423,0.839176,0.845192,0.846588,0.845192,0.845036
2,Random Forest,0.816106,0.829674,0.821635,0.822404,0.830769,0.834650,0.830769,0.830277
3,AdaBoost,0.782692,0.783154,0.782692,0.782602,0.778846,0.778846,0.778846,0.778846
4,XGBoost,0.826923,0.829410,0.826923,0.826592,0.837500,0.839217,0.837500,0.837294


In [ ]:
X = list(df['doc2vec_100'])
y = df['quality']

In [ ]:
doc2vec_100 = train_and_evaluate_models(X, y)
doc2vec_100

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.803365,0.804103,0.803365,0.803243,0.809615,0.809620,0.809615,0.809615
1,SVM,0.837500,0.839701,0.837500,0.837230,0.839423,0.840968,0.839423,0.839241
2,Random Forest,0.825240,0.826212,0.822837,0.821345,0.834615,0.841919,0.834615,0.833727
3,AdaBoost,0.783654,0.783970,0.783654,0.783595,0.778846,0.778863,0.778846,0.778843
4,XGBoost,0.825721,0.827597,0.825721,0.825471,0.830769,0.831874,0.830769,0.830628


In [ ]:
X = list(df['doc2vec_150'])
y = df['quality']

In [ ]:
doc2vec_150 = train_and_evaluate_models(X, y)
doc2vec_150

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.801683,0.802382,0.801683,0.801567,0.816346,0.816488,0.816346,0.816326
1,SVM,0.838462,0.840308,0.838462,0.838240,0.844231,0.845380,0.844231,0.844101
2,Random Forest,0.817308,0.827311,0.821875,0.812575,0.821154,0.827792,0.821154,0.820244
3,AdaBoost,0.788221,0.789305,0.788221,0.788030,0.800000,0.800444,0.800000,0.799926
4,XGBoost,0.820673,0.823055,0.820673,0.820337,0.836538,0.837517,0.836538,0.836420


In [ ]:
X = list(df['doc2vec_200'])
y = df['quality']

In [ ]:
doc2vec_200 = train_and_evaluate_models(X, y)
doc2vec_200

,Model,CV Accuracy,CV Precision,CV Recall,CV F1 Score,Test Accuracy,Test Precision,Test Recall,Test F1 Score
0,Logistic Regression,0.810337,0.811043,0.810337,0.810225,0.822115,0.822260,0.822115,0.822095
1,SVM,0.839423,0.841855,0.839423,0.839130,0.838462,0.839592,0.838462,0.838327
2,Random Forest,0.817067,0.829906,0.819712,0.816691,0.821154,0.831531,0.821154,0.819743
3,AdaBoost,0.790385,0.791036,0.790385,0.790271,0.785577,0.785705,0.785577,0.785553
4,XGBoost,0.830769,0.832806,0.830769,0.830510,0.831731,0.834703,0.831731,0.831356
